In [1]:
pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 13.5 MB/s eta 0:00:00


In [2]:
from pulp import *
import math
import copy
import random

In [3]:
def GenerateGrid(n):
  global grid
  grid = [[0] * n**2 for i in range(n**2)]

  size=len(grid)
  bsize=int(math.sqrt(len(grid)))
  index_list=[]
  for i in range(1,size+1):
      list = []
      for j in range(1,size+1):
          index = (i,j)
          list.append(index)
      index_list.append(list)
  box_list=[]
  for i in range(0,bsize):
      for j in range(0,bsize):
          box = [index_list[bsize*i+p][bsize*j+q] for p in range(0,bsize) for q in range(0,bsize)]
          box_list.append(box)

  lp = LpProblem("Sudoku LP",LpMinimize)
  rows = range(1,size+1)
  cols = range(1,size+1)
  nums = range(1,size+1)

  x_ijk = LpVariable.dicts("x",(rows,cols,nums),0,1,LpInteger)
  obj= lpSum(x_ijk[i][j][k] for i in rows for j in cols for k in nums)
  lp += obj, "Objective Function"

  for j in cols:
      for k in nums:
          lp += lpSum([x_ijk[i][j][k] for i in rows]) == 1,""
  for i in rows:
      for k in nums:
          lp += lpSum([x_ijk[i][j][k] for j in cols]) == 1,""
  for i in rows:
      for j in cols:
          lp += lpSum([x_ijk[i][j][k] for k in nums]) == 1,""
  for k in nums:
      for box in box_list:
          lp += lpSum([x_ijk[i][j][k] for (i,j) in box]) == 1,""
  for i in range(len(grid)):
      for j in range(len(grid)):
          if grid[i][j]==0:
            continue
          else:
            lp += x_ijk[i+1][j+1][grid[i][j]] == 1,""

  for a in range(1,random.randint(1, n)+1):
    numbers = [*range(1,size+1)]
    random.shuffle(numbers)
    lp.solve()
    for i in range(len(grid)):
      for j in range(len(grid)):
        for k in range(len(grid)):
          if x_ijk[i+1][j+1][k+1].varValue==1:
            grid[i][j] = numbers[k]
          else:
            continue
    lp += lpSum(x_ijk[i][j][k] for k in nums for i in rows for j in cols if value(x_ijk[i][j][k]) == 1) <= (len(grid)**2-1)

  print("Solusi: ")
  al = "╔"; am = "╦"; ar = "╗"; g = "═"
  ml = "╠"; mm = "╬"; mr = "╣"
  bl = "╚"; bm = "╩"; br = "╝"
  pgaris = (bsize*2+1)*g
  for i in rows:
    line=""
    if i==1: print(al+(bsize-1)*(pgaris+am)+(pgaris+ar))
    elif i in range(1+bsize,size,bsize): print(ml+(bsize-1)*(pgaris+mm)+(pgaris+mr))
    for j in cols:
      p = grid[i-1][j-1]
      if j in range(1,size,bsize): line+="║ "
      line += str(p)+" "
      if j == size: line+="║ "
    print(line)
  print(bl+(bsize-1)*(pgaris+bm)+(pgaris+br))


In [4]:
GenerateGrid(2)

Solusi: 
╔═════╦═════╗
║ 2 1 ║ 3 4 ║ 
║ 4 3 ║ 2 1 ║ 
╠═════╬═════╣
║ 3 4 ║ 1 2 ║ 
║ 1 2 ║ 4 3 ║ 
╚═════╩═════╝


/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [5]:
def UniqueTest(grid):
    global feasible_solution_count
    size=len(grid)
    bsize=int(math.sqrt(len(grid)))
    index_list=[]
    for i in range(1,size+1):
        list = []
        for j in range(1,size+1):
            index = (i,j)
            list.append(index)
        index_list.append(list)
    box_list=[]
    for i in range(0,bsize):
        for j in range(0,bsize):
            box = [index_list[bsize*i+p][bsize*j+q] for p in range(0,bsize) for q in range(0,bsize)]
            box_list.append(box)

    lp = LpProblem("Sudoku LP",LpMinimize)
    rows = range(1,size+1)
    cols = range(1,size+1)
    nums = range(1,size+1)

    x_ijk = LpVariable.dicts("x",(rows,cols,nums),0,1,LpInteger)
    obj=lpSum(x_ijk[i][j][k] for i in rows for j in cols for k in nums)
    lp += obj, "Objective Function"

    for j in cols:
        for k in nums:
            lp += lpSum([x_ijk[i][j][k] for i in rows]) == 1,""
    for i in rows:
        for k in nums:
            lp += lpSum([x_ijk[i][j][k] for j in cols]) == 1,""
    for i in rows:
        for j in cols:
            lp += lpSum([x_ijk[i][j][k] for k in nums]) == 1,""
    for j in nums:
        for box in box_list:
            lp += lpSum([x_ijk[i][j][k] for (i,j) in box]) == 1,""
    for i in range(len(grid)):
        for j in range(len(grid)):
            if grid[i][j]==0:
              continue
            else:
              lp += x_ijk[i+1][j+1][grid[i][j]] == 1,""
    feasible_solution_count = 0
    while True:
      lp.solve()
      if str(LpStatus[lp.status]) == "Infeasible":
        break
      else:
        lp += lpSum(x_ijk[i][j][k] for k in nums for i in rows for j in cols if value(x_ijk[i][j][k]) == 1) <= (len(grid)**2-1)
        feasible_solution_count += 1
        if feasible_solution_count == 3:
          break

In [6]:
def GeneratePuzzle(n):
  global newgrid
  global step_list
  global tempgrid1

  GenerateGrid(n)

  size=len(grid)
  bsize=int(math.sqrt(len(grid)))
  rows = range(1,size+1)
  cols = range(1,size+1)
  nums = range(1,size+1)

  newgrid = copy.deepcopy(grid)
  listofindex = []
  for i in range(len(grid)):
    for j in range(len(grid)):
      index = (i,j)
      listofindex.append(index)
  random.shuffle(listofindex)
  x = 0
  step_list = []

  while x != len(listofindex):
    i,j = listofindex[x]
    tempgrid = copy.deepcopy(newgrid)
    tempgrid1 = copy.deepcopy(newgrid)
    tempgrid1[i][j] = 0
    UniqueTest(tempgrid1)
    x += 1
    if feasible_solution_count==1:
      step_list.append(tempgrid1)
      newgrid = copy.deepcopy(tempgrid1)
    else:
      newgrid = copy.deepcopy(tempgrid)
      continue

  print("Masalah: ")
  al = "╔"; am = "╦"; ar = "╗"; g = "═"
  ml = "╠"; mm = "╬"; mr = "╣"
  bl = "╚"; bm = "╩"; br = "╝"
  pgaris = (bsize*2+1)*g
  for i in rows:
    line=""
    if i==1: print(al+(bsize-1)*(pgaris+am)+(pgaris+ar))
    elif i in range(1+bsize,size,bsize): print(ml+(bsize-1)*(pgaris+mm)+(pgaris+mr))
    for j in cols:
      p = newgrid[i-1][j-1]
      if j in range(1,size,bsize): line+="║ "
      line += str(p)+" "
      if j == size: line+="║ "
    print(line)
  print(bl+(bsize-1)*(pgaris+bm)+(pgaris+br))

In [7]:
print(GeneratePuzzle(2))

Solusi: 
╔═════╦═════╗
║ 4 3 ║ 1 2 ║ 
║ 2 1 ║ 4 3 ║ 
╠═════╬═════╣
║ 1 2 ║ 3 4 ║ 
║ 3 4 ║ 2 1 ║ 
╚═════╩═════╝
Masalah: 
╔═════╦═════╗
║ 4 0 ║ 0 2 ║ 
║ 2 0 ║ 0 0 ║ 
╠═════╬═════╣
║ 0 0 ║ 3 0 ║ 
║ 0 0 ║ 0 0 ║ 
╚═════╩═════╝
None


In [8]:
print(GeneratePuzzle(3))

Solusi: 
╔═══════╦═══════╦═══════╗
║ 4 6 3 ║ 7 5 1 ║ 2 9 8 ║ 
║ 7 1 8 ║ 3 9 2 ║ 6 5 4 ║ 
║ 5 9 2 ║ 8 6 4 ║ 7 3 1 ║ 
╠═══════╬═══════╬═══════╣
║ 9 8 7 ║ 4 1 6 ║ 3 2 5 ║ 
║ 6 2 4 ║ 5 7 3 ║ 1 8 9 ║ 
║ 1 3 5 ║ 2 8 9 ║ 4 6 7 ║ 
╠═══════╬═══════╬═══════╣
║ 3 5 1 ║ 6 4 8 ║ 9 7 2 ║ 
║ 2 7 9 ║ 1 3 5 ║ 8 4 6 ║ 
║ 8 4 6 ║ 9 2 7 ║ 5 1 3 ║ 
╚═══════╩═══════╩═══════╝
Masalah: 
╔═══════╦═══════╦═══════╗
║ 0 6 3 ║ 7 0 0 ║ 2 9 0 ║ 
║ 0 0 0 ║ 0 0 2 ║ 0 5 0 ║ 
║ 5 0 0 ║ 0 0 4 ║ 0 0 0 ║ 
╠═══════╬═══════╬═══════╣
║ 9 0 0 ║ 0 1 0 ║ 0 2 5 ║ 
║ 6 2 0 ║ 0 0 0 ║ 0 0 0 ║ 
║ 0 0 5 ║ 0 8 0 ║ 0 0 7 ║ 
╠═══════╬═══════╬═══════╣
║ 0 0 0 ║ 0 4 8 ║ 9 7 2 ║ 
║ 2 0 9 ║ 0 3 0 ║ 0 0 6 ║ 
║ 0 4 6 ║ 0 0 7 ║ 0 1 0 ║ 
╚═══════╩═══════╩═══════╝
None


In [9]:
print(GeneratePuzzle(4))

Solusi: 
╔═════════╦═════════╦═════════╦═════════╗
║ 6 14 9 16 ║ 4 10 8 12 ║ 5 2 7 13 ║ 11 3 1 15 ║ 
║ 13 5 10 15 ║ 16 7 1 2 ║ 9 12 3 11 ║ 8 6 4 14 ║ 
║ 2 8 12 7 ║ 9 3 14 11 ║ 1 4 6 15 ║ 5 10 16 13 ║ 
║ 4 11 3 1 ║ 5 13 15 6 ║ 8 14 10 16 ║ 2 12 7 9 ║ 
╠═════════╬═════════╬═════════╬═════════╣
║ 10 6 11 4 ║ 2 15 3 16 ║ 7 13 9 5 ║ 1 14 12 8 ║ 
║ 15 7 1 12 ║ 8 6 10 9 ║ 2 11 14 4 ║ 3 16 13 5 ║ 
║ 16 3 8 14 ║ 13 4 11 5 ║ 10 15 1 12 ║ 9 7 6 2 ║ 
║ 9 13 5 2 ║ 14 12 7 1 ║ 3 8 16 6 ║ 10 11 15 4 ║ 
╠═════════╬═════════╬═════════╬═════════╣
║ 8 12 15 10 ║ 11 9 4 14 ║ 16 7 5 2 ║ 13 1 3 6 ║ 
║ 1 4 7 9 ║ 15 16 13 8 ║ 14 6 11 3 ║ 12 2 5 10 ║ 
║ 5 16 13 6 ║ 10 1 2 3 ║ 12 9 15 8 ║ 7 4 14 11 ║ 
║ 11 2 14 3 ║ 12 5 6 7 ║ 13 10 4 1 ║ 15 8 9 16 ║ 
╠═════════╬═════════╬═════════╬═════════╣
║ 12 15 6 5 ║ 7 8 16 13 ║ 11 3 2 14 ║ 4 9 10 1 ║ 
║ 14 1 4 11 ║ 3 2 12 10 ║ 6 5 13 9 ║ 16 15 8 7 ║ 
║ 7 9 2 8 ║ 1 14 5 4 ║ 15 16 12 10 ║ 6 13 11 3 ║ 
║ 3 10 16 13 ║ 6 11 9 15 ║ 4 1 8 7 ║ 14 5 2 12 ║ 
╚═════════╩═════════╩══